In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Feature engineering function
def engineer_features_no_pca(df):
    df = df.copy()
    new_cols = {}

    for comp in range(1, 6):
        for prop in range(1, 11):
            prop_col = f'Component{comp}_Property{prop}'
            frac_col = f'Component{comp}_fraction'
            new_cols[f'{prop_col}_weighted'] = df[prop_col] * df[frac_col]

    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]
            new_cols[f'{frac_i}_div_{frac_j}'] = df[frac_i] / (df[frac_j] + 1e-8)

    total_fraction = sum(df[f'Component{i}_fraction'] for i in range(1, 6))
    new_cols['Total_fraction'] = total_fraction
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_to_total'] = df[frac_col] / (total_fraction + 1e-8)

    for comp in range(1, 6):
        props = [f'Component{comp}_Property{p}' for p in range(1, 11)]
        new_cols[f'Component{comp}_prop_mean'] = df[props].mean(axis=1)
        new_cols[f'Component{comp}_prop_std'] = df[props].std(axis=1)
        new_cols[f'Component{comp}_prop_min'] = df[props].min(axis=1)
        new_cols[f'Component{comp}_prop_max'] = df[props].max(axis=1)
        new_cols[f'Component{comp}_prop_range'] = new_cols[f'Component{comp}_prop_max'] - new_cols[f'Component{comp}_prop_min']

    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_squared'] = df[frac_col] ** 2

    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]

    new_features_df = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, new_features_df], axis=1)
    return df

# Load data
X = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_train.csv')
y = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/y_train.csv')
X_test = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_test.csv')

# Feature engineering
X_fe = engineer_features_no_pca(X)
X_test_fe = engineer_features_no_pca(X_test)

# Top 20 features for Target 10
top20_features_target10 = [
    'Component4_fraction',
    'Component4_fraction_x_Component5_fraction',
    'Component2_fraction_div_Component4_fraction',
    'Component3_Property10_weighted',
    'Component2_fraction_div_Component5_fraction',
    'Component1_Property10_weighted',
    'Component3_Property10',
    'Component2_Property10_weighted',
    'Component4_Property10_weighted',
    'Component5_fraction',
    'Component4_Property10',
    'Component1_fraction_x_Component2_fraction',
    'Component2_fraction',
    'Component5_Property10_weighted',
    'Component2_Property10',
    'Component1_Property10',
    'Component5_Property10',
    'Component3_fraction_x_Component4_fraction',
    'Component1_fraction_div_Component4_fraction',
    'Component1_fraction_div_Component5_fraction'
]

# Filter to top 20 features
X_fe = X_fe[top20_features_target10]
X_test_fe = X_test_fe[top20_features_target10]

# Train/val split
target_index = 9
X_train, X_val, y_train, y_val = train_test_split(X_fe, y, test_size=0.2, random_state=42)

# Model setup
model = xgb.XGBRegressor(
    max_depth=1,
    n_estimators=100000,
    learning_rate=0.05,
    reg_alpha=0.5,
    reg_lambda=3.5,
    eval_metric='mae',
    early_stopping_rounds=50,
    random_state=42,
    verbosity=1
)

print(f"\nTraining Target {target_index + 1} using TOP 20 features...")
model.fit(
    X_train, y_train.iloc[:, target_index],
    eval_set=[(X_val, y_val.iloc[:, target_index])],
    verbose=50
)

# Predictions
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)
test_preds = model.predict(X_test_fe)

# Evaluation
print(f"\nTrain MAE: {mean_absolute_error(y_train.iloc[:, target_index], train_preds):.5f}")
print(f"Validation MAE: {mean_absolute_error(y_val.iloc[:, target_index], val_preds):.5f}")
print(f"Train MAPE: {mean_absolute_percentage_error(y_train.iloc[:, target_index], train_preds):.5f}")
print(f"Validation MAPE: {mean_absolute_percentage_error(y_val.iloc[:, target_index], val_preds):.5f}")

# Save test predictions
pd.DataFrame(test_preds, columns=[f'Target{target_index + 1}_prediction']).to_csv(f'test_predictions_target_{target_index + 1}.csv', index=False)
print(f"\nSaved test predictions to test_predictions_target_{target_index + 1}.csv")



Training Target 10 using TOP 20 features...
[0]	validation_0-mae:0.81170
[50]	validation_0-mae:0.44513
[100]	validation_0-mae:0.34243
[150]	validation_0-mae:0.29287
[200]	validation_0-mae:0.25581
[250]	validation_0-mae:0.22925
[300]	validation_0-mae:0.20932
[350]	validation_0-mae:0.19437
[400]	validation_0-mae:0.18191
[450]	validation_0-mae:0.17174
[500]	validation_0-mae:0.16376
[550]	validation_0-mae:0.15698
[600]	validation_0-mae:0.15125
[650]	validation_0-mae:0.14618
[700]	validation_0-mae:0.14162
[750]	validation_0-mae:0.13780
[800]	validation_0-mae:0.13442
[850]	validation_0-mae:0.13179
[900]	validation_0-mae:0.12940
[950]	validation_0-mae:0.12747
[1000]	validation_0-mae:0.12538
[1050]	validation_0-mae:0.12344
[1100]	validation_0-mae:0.12152
[1150]	validation_0-mae:0.11984
[1200]	validation_0-mae:0.11811
[1250]	validation_0-mae:0.11646
[1300]	validation_0-mae:0.11504
[1350]	validation_0-mae:0.11359
[1400]	validation_0-mae:0.11207
[1450]	validation_0-mae:0.11076
[1500]	validation_